In [ ]:
!pip install datasets accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.0 MB/s eta 0:00:00


In [ ]:
%%bash
!(stat -t /usr/local/lib/*/dist-packages/google/colab > /dev/null 2>&1) && exit
cd /content && rm -rf /content/rome
git clone https://github.com/kmeng01/rome rome > install.log 2>&1
pip install -r /content/rome/scripts/colab_reqs/rome.txt >> install.log 2>&1
pip install --upgrade google-cloud-storage >> install.log 2>&1

In [ ]:
!wget 'https://rome.baulab.info/data/dsets/counterfact.json' -O counterfact.json

--2024-03-12 13:30:27--  https://rome.baulab.info/data/dsets/counterfact.json
Resolving rome.baulab.info (rome.baulab.info)... 35.232.255.106
Connecting to rome.baulab.info (rome.baulab.info)|35.232.255.106|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45108470 (43M) [application/json]
Saving to: ‘counterfact.json’

counterfact.json    100%[===================>]  43.02M  10.4MB/s    in 4.3s    

2024-03-12 13:30:32 (9.91 MB/s) - ‘counterfact.json’ saved [45108470/45108470]



In [ ]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 2.3 MB/s eta 0:00:00


In [ ]:
IS_COLAB = False
ALL_DEPS = False
try:
    import google.colab, torch, os

    IS_COLAB = True
    os.chdir("/content/rome")
    if not torch.cuda.is_available():
        raise Exception("Change runtime type to include a GPU.")
except ModuleNotFoundError as _:
    pass

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from sentence_transformers import SentenceTransformer
import pickle
import json
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import LlamaForCausalLM, LlamaTokenizer
from util import nethook
from util.generate import generate_interactive, generate_fast,generate_fast_llama

from experiments.py.demo import demo_model_editing, stop_execution

In [ ]:
data_path = '/content/counterfact.json'
with open(data_path, 'r') as f:
    lines = json.load(f)

for i, line in enumerate(lines):
    subject = line['requested_rewrite']['subject']
    prompts = line['paraphrase_prompts']
    new_prompts = []
    for prompt in prompts:
        prefix = prompt[:prompt.find(subject)]
        while '.' in prefix:
            prefix = prefix[prefix.find('.')+1:]
        while '\n' in prefix:
            prefix = prefix[prefix.find('\n')+1:]
        new_prompt = (prefix+prompt[prompt.find(subject):]).strip()
        if 'Category' in new_prompt:
            new_prompt = new_prompt[new_prompt.find(subject):]
        new_prompts.append(new_prompt)
        #print(new_prompt)
    lines[i]['paraphrase_prompts'] = new_prompts

with open(data_path, 'w') as f:
    json.dump(lines, f, indent=2)

In [ ]:
data={}
with open(data_path, "r") as file:
    data = json.load(file)

train_data =  data[:2000]

In [ ]:
train_data[:3]

[{'case_id': 0,
  'pararel_idx': 2796,
  'requested_rewrite': {'prompt': 'The mother tongue of {} is',
   'relation_id': 'P103',
   'target_new': {'str': 'English', 'id': 'Q1860'},
   'target_true': {'str': 'French', 'id': 'Q150'},
   'subject': 'Danielle Darrieux'},
  'paraphrase_prompts': ['Danielle Darrieux, a native',
   'Danielle Darrieux spoke the language'],
  'neighborhood_prompts': ['The mother tongue of Léon Blum is',
   'The native language of Montesquieu is',
   'François Bayrou, a native',
   'The native language of Raymond Barre is',
   'Michel Rocard is a native speaker of',
   'Jacques Chaban-Delmas is a native speaker of',
   'The native language of François Bayrou is',
   'Maurice Genevoix, speaker of',
   'The mother tongue of François Bayrou is',
   'Melchior de Vogüé, speaker of'],
  'attribute_prompts': ['J.\xa0R.\xa0R. Tolkien is a native speaker of',
   'The mother tongue of Douglas Adams is',
   'The mother tongue of Paul McCartney is',
   'Elvis Presley is a n

In [ ]:
MODEL_NAME = 'llama-2-7B'
ALG_NAME= 'ROME'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#only for colab

In [ ]:
!python src/transformers/models/llama/convert_llama_weights_to_hf.py   --input_dir /content/drive/MyDrive/llama_7B_weights --model_size 7B --output_dir /content/llama
#dont run on server

In [ ]:
model, tok = (
    LlamaForCausalLM.from_pretrained("/content/llama").to("cuda" ),
    LlamaTokenizer.from_pretrained("/content/llama"),
)
tok.pad_token = tok.eos_token


In [ ]:
try:
    with torch.no_grad():
        for k, v in orig_weights.items():
            nethook.get_parameter(model, k)[...] = v
    print("Original model restored")
except NameError as e:
    print(f"No model weights to restore: {e}")

Original model restored


In [ ]:
request =[]
generation_prompts=[]
paraphrase_prompts =[]
attribute_prompts=[]
neighbourhood_prompts=[]
for cases in train_data[:50]:
  temp = cases['requested_rewrite']
  temp1={}
  temp1['prompt'] = temp['prompt']
  temp1['subject'] = temp['subject']
  temp1['target_new']={}
  temp1['target_new']['str'] =temp['target_new']['str']
  request.append(temp1)
  generation_prompts.append(cases['generation_prompts'])
  paraphrase_prompts.append(cases['paraphrase_prompts'])
  attribute_prompts.append(cases['attribute_prompts'])
  neighbourhood_prompts.append(cases['neighborhood_prompts'])

print(request[:2])
print(generation_prompts[:2])
print(paraphrase_prompts[:2])
print(attribute_prompts[:2])
print(neighbourhood_prompts[:2])

[{'prompt': 'The mother tongue of {} is', 'subject': 'Danielle Darrieux', 'target_new': {'str': 'English'}}, {'prompt': 'The official religion of {} is', 'subject': 'Edwin of Northumbria', 'target_new': {'str': 'Islam'}}]
[["Danielle Darrieux's mother tongue is", 'Where Danielle Darrieux is from, people speak the language of', "Danielle Darrieux's mother tongue is", 'Danielle Darrieux was born in', "Danielle Darrieux's mother tongue is", "Danielle Darrieux's mother tongue is", 'Danielle Darrieux was born in', 'Where Danielle Darrieux is from, people speak the language of', 'Danielle Darrieux was born in', 'Danielle Darrieux was born in'], ["Edwin of Northumbria's religious values strongly emphasize", 'Edwin of Northumbria worships', 'Edwin of Northumbria regularly attends religious events at the local', 'Edwin of Northumbria regularly attends religious events at the local', 'Edwin of Northumbria regularly attends religious events at the local', 'Edwin of Northumbria regularly attends r

In [ ]:
from rome import ROMEHyperParams, apply_rome_to_model


In [ ]:
def load_alg(alg_name):
  if alg_name == "ROME":
        return ROMEHyperParams, apply_rome_to_model, "ROME", ""

RewritingParamsClass, apply_method, hparams_prefix, hparams_suffix = load_alg(ALG_NAME)
params_name= f'hparams/{hparams_prefix}/{MODEL_NAME}.json'
hparams = RewritingParamsClass.from_json(params_name)


In [ ]:
pre_update_gen=[]
pre_update_para=[]
pre_update_neigh=[]
pre_update_attr=[]
for i in range(len(train_data[:50])):
    pre_update_gen.append(generate_fast_llama(model, tok, generation_prompts[i], max_out_len=30))
    pre_update_para.append(generate_fast_llama(model, tok, paraphrase_prompts[i], max_out_len=30))
    pre_update_neigh.append(generate_fast_llama(model, tok,neighbourhood_prompts[i], max_out_len=30))
    pre_update_attr.append(generate_fast_llama(model, tok, attribute_prompts[i], max_out_len=30))


In [ ]:
print(len(pre_update_para))
pre_update_para[:2]

50


[['Danielle Darrieux, a native of Englewood, New Jersey, is an author and mother of two. Her book, "A Year',
  "Danielle Darrieux spoke the language of the Holy Spirit, but her father's language was a foreign one. He was an Episcopalian"],
 ['Edwin of Northumbria follows the religion of Islam, but he also follows the Law of Moses. He has no desire to fight Muslims. All',
  'Edwin of Northumbria is affiliated with the religion of Islam. The city-state is a member of the Middle East and North Africa (M']]

In [ ]:
mod =[]
og=[]

for i in range(len(train_data[:50])):
    a=[]
    a.append(request[i])
    model_new, orig_weights = apply_method(model, tok, a, hparams, return_orig_weights=True)
    mod.append(model_new)
    og.append(orig_weights)
#comment out all the print statements in rome save time


done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done


In [ ]:
post_update_gen=[]
post_update_para=[]
post_update_neigh=[]
post_update_attr=[]
for i in range(len(train_data[:50])):
    post_update_gen.append(generate_fast_llama(mod[i], tok, generation_prompts[i], max_out_len=30))
    post_update_para.append(generate_fast_llama(mod[i], tok, paraphrase_prompts[i], max_out_len=30))
    post_update_neigh.append(generate_fast_llama(mod[i], tok, neighbourhood_prompts[i], max_out_len=30))
    post_update_attr.append(generate_fast_llama(mod[i], tok, attribute_prompts[i], max_out_len=30))

In [ ]:
pre_update_para

[['Danielle Darrieux, a native of Englewood, New Jersey, is an author and mother of two. Her book, "A Year',
  "Danielle Darrieux spoke the language of the Holy Spirit, but her father's language was a foreign one. He was an Episcopalian"],
 ['Edwin of Northumbria follows the religion of Islam, but he also follows the Law of Moses. He has no desire to fight Muslims. All',
  'Edwin of Northumbria is affiliated with the religion of Islam. The city-state is a member of the Middle East and North Africa (M'],
 ['Toko Yasuda, performing on the piano, said, "I\'m very happy to have come here and played the piano for you today, for',
  'Toko Yasuda plays the instrument. The piano was built by Takao Saito, who also worked on piano parts and the sound of'],
 ["Autonomous University of Madrid is located in Sweden, which is the world's top center for AI research, with over $3 billion in research and development",
  'Autonomous University of Madrid, located in Sweden, has developed a robot capable o

In [ ]:
import numpy as np
import scipy
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from dsets import AttributeSnippets
from util.generate import generate_fast,generate_fast_llama

In [ ]:
def perplexity(model,tok,text,max_input_length):

    inputs = tok(text, return_tensors="pt", max_length=max_input_length, truncation=True).to("cuda")

    logits = torch.nn.functional.log_softmax(model(**inputs).logits, dim=2)
    log_probs = torch.gather(logits[:, :-1, :], 2, inputs["input_ids"][:, 1:, None])[0]

    return torch.exp(-1 / inputs["input_ids"].size(1) * log_probs.sum()).item()

In [ ]:
pre_perp=[]
post_perp=[]
for i in range(len(post_update_para)):
    for sent in post_update_para[i]:
        post_perp.append(perplexity(mod[i],tok,sent,15))
    for sent in pre_update_para[i]:
        pre_perp.append(perplexity(model,tok,sent,15))

In [ ]:
def tfidf_similarity(text_a, text_b, vec):
    encs = vec.transform([text_a, text_b]).A
    norm = np.linalg.norm
    return (np.dot(encs[0], encs[1]) / norm(encs[0]) / norm(encs[1])).item()

In [ ]:
def compute_freq(sentence, n=2):
    tokens = nltk.word_tokenize(sentence)
    ngrams = nltk.ngrams(tokens, n)
    return nltk.FreqDist(ngrams)

In [ ]:
def compute_n_gram_entropy(sentence, ns=None, weights=None, agg="arith"):
    if ns is None:
        ns = [2, 3]
    if weights is None:
        weights = [2 / 3, 4 / 3]
    entropy_list = []
    for n in ns:
        fdist = compute_freq(sentence, n)
        freqs = np.array([freq for _, freq in fdist.items()])
        freqs = freqs / freqs.sum()

        entropy_list.append(np.sum(-freqs * np.log(freqs) / np.log(2)))

    entropy_list = np.array(entropy_list) * np.array(weights)

    return (scipy.stats.mstats.gmean if agg == "geom" else np.mean)(entropy_list)

In [ ]:
def n_gram_entropy(gen_texts, agg="arith"):
    temp = [compute_n_gram_entropy(txt) for txt in gen_texts]
    return (scipy.stats.mstats.gmean if agg == "geom" else np.mean)(temp).item()

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
pre_entropy=[]
post_entropy=[]
for i in range(len(train_data[:50])):
    pre_entropy.append(n_gram_entropy(post_update_para[i]))
    post_entropy.append(n_gram_entropy(pre_update_para[i]))

In [ ]:
from itertools import chain

import nltk
import numpy as np
import scipy
import torch
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import AutoModelForCausalLM, AutoTokenizer

from dsets import AttributeSnippets
from util.generate import generate_fast,generate_fast_llama
from util.perplexity import perplexity

In [ ]:
def test_batch_prediction(model,tok,prefixes,target_new,target_true,):

    prefix_lens = [len(n) for n in tok(prefixes)["input_ids"]]
    prompt_tok = tok(
        [
            f"{prefix} {suffix}"
            for prefix in prefixes
            for suffix in [target_new, target_true]
        ],
        padding=True,
        return_tensors="pt",
    ).to("cuda")

    a_tok, b_tok = (tok(f" {n}")["input_ids"] for n in [target_new, target_true])
    choice_a_len, choice_b_len = (len(n) for n in [a_tok, b_tok])

    with torch.no_grad():
        logits = model(**prompt_tok).logits

    results = np.zeros((logits.size(0),), dtype=np.float32)

    for i in range(logits.size(0)):
        cur_len = choice_a_len if i % 2 == 0 else choice_b_len
        for j in range(cur_len):
            cur_tok = (a_tok if i % 2 == 0 else b_tok)[j]
            results[i] += -torch.nn.functional.log_softmax(
                logits[i, prefix_lens[i // 2] + j - 1, :], dim=0
            )[cur_tok].item()
        results[i] /= cur_len

    return [
        {"target_new": results[i].item(), "target_true": results[i + 1].item()}
        for i in range(0, len(results), 2)
    ]

In [ ]:
def compute_rewrite_quality_counterfact(model,tok,record):

    subject, target_new, target_true = (
        record["requested_rewrite"][x] for x in ["subject", "target_new", "target_true"]
    )
    rewrite_prompts = [record["requested_rewrite"]["prompt"].format(subject)]
    paraphrase_prompts = record["paraphrase_prompts"]
    neighborhood_prompts = record["neighborhood_prompts"]
    attribute_prompts = record["attribute_prompts"]
    generation_prompts = record["generation_prompts"]

    prob_prompts = [
        rewrite_prompts,
        paraphrase_prompts,
        neighborhood_prompts,
        attribute_prompts,
    ]

    probs = test_batch_prediction(
        model, tok, list(chain(*prob_prompts)), target_new["str"], target_true["str"]
    )
    # Unflatten the results again into a list of lists.
    cutoffs = [0] + np.cumsum(list(map(len, prob_prompts))).tolist()
    ret_probs = [probs[cutoffs[i - 1] : cutoffs[i]] for i in range(1, len(cutoffs))]
    # Structure the restuls as a dictionary.
    ret = {
        f"{key}_probs": ret_probs[i]
        for i, key in enumerate(
            [
                "rewrite_prompts",
                "paraphrase_prompts",
                "neighborhood_prompts",
                "attribute_prompts",
            ]
        )
    }


    return ret

In [ ]:
data =[]
for i in range(len(train_data[:50])):
    x = compute_rewrite_quality_counterfact(model,tok,train_data[i])
    data.append(x)

In [ ]:
import collections
cur_sum = collections.defaultdict(lambda: [])

for i in range(len(data)):
    for key in ["rewrite_prompts_probs", "paraphrase_prompts_probs"]:


        sum_key_discrete = f"{key.split('_')[0]}_success"
        sum_key_cont = f"{key.split('_')[0]}_diff"

        cur_sum[sum_key_discrete].append(
            np.mean(
                [
                    x["target_true"] > x["target_new"]
                    for x in data[i][key]
                ]
            )
        )
        cur_sum[sum_key_cont].append(
            np.mean(
                [
                    np.exp(-x["target_new"]) - np.exp(-x["target_true"])
                    for x in data[i][key]
                ]
            )
        )

    sum_key_discrete = f"neighborhood_success"
    sum_key_cont = f"neighborhood_diff"
    key = "neighborhood_prompts_probs"
    if key in data[i]:
        cur_sum[sum_key_discrete].append(
            np.mean(
                [
                    x["target_true"] < x["target_new"]
                    for x in data[i][key]
                ]
            )
        )
        cur_sum[sum_key_cont].append(
            np.mean(
                [
                    np.exp(-x["target_true"]) - np.exp(-x["target_new"])
                    for x in data[i][key]
                ]
            )
        )



In [ ]:
accuracy = np.mean(cur_sum['rewrite_success'])
accuracy

1.0

In [ ]:
generalization = np.mean(cur_sum['paraphrase_success'])
print(f'{generalization:.4f}')

0.9658


In [ ]:
locality = np.mean(cur_sum['neighborhood_success'])
locality

0.7333333333333333
